In [14]:
import cv2
import numpy as np
import tensorflow as tf
from tkinter import filedialog
from PIL import Image, ImageTk
import tkinter as tk

# Load the trained color detection model
model = tf.keras.models.load_model("color_car_detection_model.keras")
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Preprocessing function (modified for 48x48 grayscale images)
def preprocess_image(image):
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Resize the image to 48x48 as required by the model
    image_resized = cv2.resize(image_gray, (48, 48))

    # Normalize the image
    image_normalized = image_resized / 255.0

    # Expand dimensions to match the model's input shape (batch_size, 48, 48, 1)
    image_expanded = np.expand_dims(image_normalized, axis=-1)  # Adding channel dimension
    image_expanded = np.expand_dims(image_expanded, axis=0)  # Adding batch dimension

    return image_expanded

# Load the pre-trained Haar Cascade for person detection
person_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')

# Function to detect and count cars and persons based on the color detection model
def detect_and_count(image_path):
    # Load image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Process the image for color detection (car color detection)
    processed_image = preprocess_image(image)
    predictions = model.predict(processed_image)

    color_labels = ["beige", "black", "blue", "brown", "yellow", "gold", "red", "green", "white", "grey", "pink", "purple", "orange", "silver", "tan"]
    predicted_color = color_labels[np.argmax(predictions)]  # Get the predicted color
    print(predicted_color)
    # Initialize counts
    car_count = 0
    person_count = 0
    colors = {
        "blue": (0, 0, 255),  # Red rectangle for blue cars
        "other": (255, 0, 0)  # Blue rectangle for other colors
    }

    # Example: Detecting cars and assigning rectangles
    rect_color = colors["other"]  # Default color
    if predicted_color == "blue":
        rect_color = colors["blue"]
        car_count += 1

    # Draw a bounding box around the detected car (for visualization)
    cv2.rectangle(image, (50, 50), (150, 150), rect_color, 2)  # Example coordinates

    # Convert image to grayscale for person detection
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect persons using the Haar Cascade classifier
    persons = person_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in persons:
        person_count += 1
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw green rectangle around person

    # Add counts to the image
    cv2.putText(image, f"Cars: {car_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(image, f"People: {person_count}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    return image

# GUI to load and display the image
def load_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    if not file_path:
        return

    # Detect and count cars and people
    result_image = detect_and_count(file_path)

    # Convert the result image to a format suitable for Tkinter
    result_image_rgb = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
    result_image_pil = Image.fromarray(result_image_rgb)
    result_image_tk = ImageTk.PhotoImage(result_image_pil)

    # Display the result in the GUI
    result_label.config(image=result_image_tk)
    result_label.image = result_image_tk

# Initialize the GUI
root = tk.Tk()
root.title("Car Color and Person Detection")

# Create GUI components
load_button = tk.Button(root, text="Load Image", command=load_image)
load_button.pack()

result_label = tk.Label(root)
result_label.pack()

# Run the GUI loop
root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 575ms/step
green
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 832ms/step
green
